<a href="https://colab.research.google.com/github/nguyenkien1402/llamaindex-practices/blob/main/evaluation-pipeline-rag/rag_evaluation_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive
MOUNTPOINT = '/content/gdrive'
DATADIR = os.path.join(MOUNTPOINT, 'MyDrive', 'llm-poc')
drive.mount(MOUNTPOINT)

Mounted at /content/gdrive


In [2]:
!pip install -r gdrive/MyDrive/llm-doc/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.3/806.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.4/422.4 kB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.9 MB/s eta 0:00:0

In [1]:
from llama_index import download_loader
from llama_index import SimpleDirectoryReader

UnstructuredReader = download_loader('UnstructuredReader')

In [47]:
dir_reader = SimpleDirectoryReader('gdrive/MyDrive/llm-doc/data', file_extractor={
  ".pdf": UnstructuredReader(),
})
documents = dir_reader.load_data()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [9]:
import pandas as pd
from llama_index.evaluation import DatasetGenerator, RelevancyEvaluator
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
    LLMPredictor,
    Response,
    StorageContext,
    load_index_from_storage
)
from llama_index.node_parser import SimpleNodeParser
from llama_index.storage.index_store import SimpleIndexStore

import chromadb
from llama_index.vector_stores import ChromaVectorStore

import openai
from llama_index.llms import OpenAI

openai.api_key = ""

# **Create Embedding Mode**


In [4]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

model_norm = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    query_instruction="Represent this sentence for searching relevant passages: finance, investments, economics, real estate"
)

# **Initialize Service Context**

In [48]:
chroma_client.delete_collection('finance')

In [49]:
# create client and a new collection
chroma_client = chromadb.PersistentClient(path='gdrive/MyDrive/llm-doc/vector_store/')
chroma_collection = chroma_client.get_or_create_collection("finance")

node_parser = SimpleNodeParser.from_defaults(chunk_size=1024, chunk_overlap=20)

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
service_context = ServiceContext.from_defaults(llm=OpenAI(model="gpt-3.5-turbo-16k", max_tokens=512, temperature=0.1),
                                               embed_model=model_norm,
                                               node_parser=node_parser)


**Approach 1: Naive Approach with VectorIndex Only**

In [50]:
indexid = 'finance_index'
index_path = 'gdrive/MyDrive/llm-doc/index'
try:
  ## load index from storage
  print(f"Load {indexid} from local path")
  storage_context = StorageContext.from_defaults(vector_store=vector_store,
                                                  persist_dir=index_path)
  index = load_index_from_storage(storage_context=storage_context, index_id=indexid)
except Exception as e:
  print(str(e))
  print("Creating new index")
  storage_context = StorageContext.from_defaults(vector_store=vector_store)
  index = VectorStoreIndex.from_documents(documents, service_context=service_context, storage_context=storage_context,)
  index.set_index_id(indexid)
  index.storage_context.persist(persist_dir=index_path)

Load finance_index from local path
[Errno 2] No such file or directory: '/content/gdrive/MyDrive/llm-doc/index/docstore.json'
Creating new index


In [51]:
query_engine = index.as_query_engine()
response = query_engine.query("what is biggest revenue?")

In [57]:
query_engine.query("How does the Nvidia's revenue in this report compare to the previous fiscal year, and what factors contributed to any changes?").response

"NVIDIA's revenue in this report has increased compared to the previous fiscal year. The revenue for the second quarter of fiscal year 2024 was $13.51 billion, which is a 101% increase from the same period in the previous year. The revenue for the first half of fiscal year 2024 was $20.70 billion, a 38% increase from the first half of the previous year. \n\nThe increase in revenue can be attributed to several factors. In the Compute & Networking segment, the growth was primarily driven by higher Data Center revenue, with Compute GPUs growing 208% year-on-year and 112% compared to the first half of fiscal year 2023. This growth was led by demand for NVIDIA HGX platform based on the Hopper and Ampere GPU architecture for large language models and generative AI. Networking also saw strong growth, up 94% year-on-year and 63% compared to the first half of last year, driven by growth in InfiniBand infrastructure to support the HGX platform.\n\nIn the Graphics segment, the increase in revenue

In [53]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='e5f3093a-4016-4116-aa73-8a44bec69856', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='dbc5ffef-826f-4e0d-bbf7-fa4575020ee7', node_type=None, metadata={}, hash='814858a2fac11b608da3b051bec03e26c3e3d2977d928cf07ee1540155774944'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='0ce41904-e255-4280-bcc1-0259c568ae4e', node_type=None, metadata={}, hash='c7cca3387eb58cd3fcf2fadd803b3d0a10040a9687052b25b8b91d62b086bd3d'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='44d40f6e-e5d4-40ab-af4d-7f1034e5e039', node_type=None, metadata={}, hash='8aa5e94f317d52971cbf625d76e584b84af2715cc893fe5200079b5261dace61')}, hash='44136fa355b3678a1146ad16f7e8649e94fb4fc21fe77e8310c060f61caaff8a', text='94\n\nPART II Item 8\n\nRevenue, classified by significant product and service offerings, was as follows:\n\n(In millions)\n\nYear Ended June 